In [62]:
import os
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from lsst.daf.persistence import Butler
import fakeInjectionPipeline as fip
import data_process_utils as dpu

from scipy.stats import norm
import scipy.stats as stats

import imp
imp.reload(fip) 

<module 'fakeInjectionPipeline' from '/global/cscratch1/sd/shl159/projects/dia_improvement/scripts/fakeInjectionPipeline.py'>

In [63]:
!pwd

/global/cscratch1/sd/shl159/projects/dia_improvement/scripts


In [64]:
def get_calexp_injection_dict(patch_list, host_mag_range_list):
    
    conn = sqlite3.connect('../data/table/gal_4639/injection_coord.sqlite')
    calexp_info_dict = {}
    injection_coord_dict = {}

    for patch in patch_list:
        patch_1, patch_2 = patch[0], patch[1]
        


        for host_mag in host_mag_range_list:
            min_mag = host_mag[0]
            max_mag = host_mag[1]

            calexp_info_query = (f'SELECT distinct(visit), filter, detector FROM injection_coord WHERE patch_1={patch_1} AND '
                                 f'patch_2={patch_2} AND min_host_mag = {min_mag} AND max_host_mag = {max_mag}'
                                )
            calexp_info = pd.read_sql_query(calexp_info_query, conn)

            calexp_info_dict[f'{patch}_{min_mag}_{max_mag}'] = calexp_info

        for host_mag in host_mag_range_list:
            min_mag = host_mag[0]
            max_mag = host_mag[1]
            calexp_info = calexp_info_dict[f'{patch}_{min_mag}_{max_mag}']
            for idx, row in calexp_info.iterrows():
                visit = row['visit']
                filt = row['filter']
                detector = row['detector']
                coord_query = (
                    f'SELECT calexp_coord_x, calexp_coord_y FROM injection_coord '
                    f"WHERE patch_1={patch_1} AND patch_2={patch_2} AND min_host_mag = {min_mag} AND max_host_mag = {max_mag} "
                    f"AND visit = {visit} AND filter = '{filt}' AND detector = {detector}"
                )
                coord = pd.read_sql_query(coord_query, conn).to_numpy()
                injection_coord_dict[f'{patch}_{int(min_mag)}_{int(max_mag)}_{int(visit)}_{int(detector)}_{filt}'] = coord
    return calexp_info_dict, injection_coord_dict

In [65]:
!pwd

/global/cscratch1/sd/shl159/projects/dia_improvement/scripts


# injection

In [66]:
patch_list = ['00', '11', '22', '33', '44', '55', '66']
host_mag_list = ['20_21', '21_22', '22_23', '23_24', '24_25']
fake_mag_list = [20, 21, 22, 23, 24]

host_mag_range_list = [(20, 21), (21, 22), (22,23), (23, 24), (24, 25)]
calexp_info_dict, injection_coord_dict = get_calexp_injection_dict(patch_list, host_mag_range_list)



In [67]:
!pwd

/global/cscratch1/sd/shl159/projects/dia_improvement/scripts


In [68]:
fake_dir = f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/injection/perfect'

injector = fip.fakeInjectionPipeline(
    fake_dir = fake_dir, patch_list = patch_list,
    host_mag_list=host_mag_list, fake_mag_list=fake_mag_list,
    calexp_info_dict=calexp_info_dict, injection_coord_dict=injection_coord_dict
)

In [40]:
# injector.make_fake_repo()
# injector.inject_fakes(poisson=False)

In [37]:
cf_type = 'default_3sig'

config = f'./config/diffimConfig_al_{cf_type}.py'
file_path = f'../al_{cf_type}_perfect.sh'
diff_dir =  f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_{cf_type}'
# injector.get_subtraction_script(diff_dir , config=config, file_path=file_path, use_slurm=True)

# exsistance

In [1]:
"""
for patch in patch_list:
    
    for host_mag in host_mag_list:
        calexp_info = calexp_info_dict[f'{patch}_{host_mag}']
        
        for index, row in calexp_info.iterrows():
            visit = row['visit']
            filt = row['filter']
            detector = row['detector']

            for fake_mag in fake_mag_list:
                fake_mag_str = str(fake_mag).replace('.', '')

                diff_path = os.path.join(
                    diff_dir, f'{patch}_{host_mag}_{visit}_{detector}_{filt}/diff_{fake_mag_str}/deepDiff'
                )
                
                if not os.path.exists(diff_path):
                    print(diff_path)
"""
print()


In [40]:
cf_type = 'default_3sig'
diff_dir =  f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_{cf_type}'
flux_dir = (
    f'/global/cscratch1/sd/shl159/projects/dia_improvement/data'
    f'/patch_0to6/diff/perfect/al_{cf_type}/detection'
)

injector.get_detection(diff_dir=diff_dir, flux_dir=flux_dir, db_name=f'detection.sqlite')
print('done')

patch:  00
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  11
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  22
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  33
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  44
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  55
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  66
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


detection results are saved in /global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/detection/detection.sqlite
fake_src contains the fake information, and artifact contains the artifact information
done


In [16]:
flux_db_path = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/detection/detection.sqlite'
coord_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/coord'
injector.get_detection_coord(flux_db_path, coord_dir)

forced photometry coordinates are saved in /global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/coord


In [26]:
cf_type = 'default_3sig'
diff_dir =  f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_{cf_type}'
coord_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/coord'
forced_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/forced'

config = './config/forcedConfig.py'
file_path = '../forced_al_default_3sig.sh'
injector.get_forced_phot_script(diff_dir, coord_dir, forced_dir, config, file_path=file_path)

patch:  00
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  11
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  22
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  33
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  44
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  55
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  66
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


please run this file: ../forced_al_default_3sig.sh


In [69]:
coord_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/coord'
forced_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/forced'
injector.get_forced_db(coord_dir, forced_dir, db_name='forced.sqlite')